In [117]:
import numpy as np
import pandas as pd
import os
import sklearn
import glob
import Toolbox
from Toolbox import NRMSE,Bias_Calc
from scipy.stats import pearsonr

In [118]:
filepath_Val = "C:\\Users\\obazgir\\Desktop\\REFINED project\\Volumetric REFINED\\NCI_Regression\\*VAL.csv"
VAL_Files = glob.glob(filepath_Val)
filepath_Test = "C:\\Users\\obazgir\\Desktop\\REFINED project\\Volumetric REFINED\\NCI_Regression\\*TEST.csv"
Test_Files = glob.glob(filepath_Test)

In [119]:
def NRMSE(Y_Target, Y_Predict):
    Y_Target = np.array(Y_Target); Y_Predict = np.array(Y_Predict);
    Y_Target = Y_Target.reshape(len(Y_Target),1);    Y_Predict = Y_Predict.reshape(len(Y_Predict),1);    
    Y_Bar = np.mean(Y_Target)
    Nom = np.sum((Y_Predict - Y_Target)**2);    Denom = np.sum((Y_Bar - Y_Target)**2)
    MSE = np.mean((Y_Predict - Y_Target)**2);   NRMSE = np.sqrt(Nom/Denom)
    R2 = 1 - NRMSE**2
    return NRMSE, R2

In [129]:
SAVE_PATH = "C:\\Users\\obazgir\\Desktop\\REFINED project\\Volumetric REFINED\\NCI_Regression\\"
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
Res = {}
for i in range(len(VAL_Files)):
    Stack_Res= np.zeros((1,4))
    Val = pd.read_csv(VAL_Files[i])
    Test = pd.read_csv(Test_Files[i])
    Y_Train = Val['Y_Val'].to_numpy()
    X_Train = Val[['MDS','LE','LLE','ISO']].to_numpy()
    Y_Test = Test['Y_Val'].to_numpy()
    X_Test = Test[['MDS','LE','LLE','ISO']].to_numpy()
    reg = LinearRegression()
    #reg = RandomForestRegressor(max_depth=10, random_state=0)
    reg.fit(X_Train, Y_Train)
    Y_Pred = reg.predict(X_Test)
    NRMSE_par,R2_par = NRMSE(Y_Test,Y_Pred)
    CNN_PCC, p_value = pearsonr(Y_Test,Y_Pred)
    Bias = Bias_Calc(Y_Test,Y_Pred)
    Fname= VAL_Files[i].split('\\')[7]
    Cell_Name = Fname.strip('VAL.csv')
    Stack_Res[0,:] = NRMSE_par,R2_par,CNN_PCC,Bias
    PD_Res = pd.DataFrame(data = Stack_Res, columns=["NRMSE","R2","PCC","Bias"])
    Res[Cell_Name] = PD_Res
    # Save the stacked points
    Test['Stacked'] = Y_Pred
    Test.to_csv(SAVE_PATH+"Stacked_Res\\"+Cell_Name+"stacked.csv")


with open(SAVE_PATH+'Stacked.csv', 'w') as f:[f.write('{0},{1}\n'.format(key, value)) for key, value in Res.items()]

In [130]:
Res

{'786_0':      NRMSE        R2       PCC      Bias
 0  0.68845  0.526037  0.726558  0.498793,
 '498':       NRMSE        R2       PCC     Bias
 0  0.709787  0.496202  0.704829  0.51586,
 '549_ATCC':       NRMSE        R2       PCC      Bias
 0  0.672905  0.547199  0.739769  0.447348,
 'CHN':       NRMSE        R2       PCC     Bias
 0  0.679965  0.537647  0.733684  0.47868,
 'BT_549':       NRMSE       R2       PCC      Bias
 0  0.742085  0.44931  0.671418  0.532077,
 'CAKI_1':       NRMSE       R2       PCC      Bias
 0  0.708209  0.49844  0.706275  0.513947,
 'CCRF_CEM':       NRMSE        R2       PCC      Bias
 0  0.719779  0.481918  0.694249  0.522492,
 'COLO_205':       NRMSE        R2       PCC      Bias
 0  0.681804  0.535143  0.731641  0.460919,
 'DLD_1':       NRMSE        R2       PCC      Bias
 0  0.685755  0.529741  0.728807  0.468404,
 'DMS_114':       NRMSE        R2       PCC     Bias
 0  0.703957  0.504444  0.710886  0.48422,
 'DMS_273':       NRMSE        R2       PCC

In [112]:
Test.head()
Test['Stacked'] = Y_Pred
Test.head()

,Unnamed: 0,Y_Val,MDS,LE,LLE,ISO,VOL,Stacked
0,0,0.557946,0.540486,0.540677,0.524917,0.544303,0.528383,0.536828
1,1,0.473133,0.500014,0.501073,0.511223,0.502395,0.486443,0.498527
2,2,0.466189,0.492360,0.494889,0.496719,0.495329,0.477587,0.489585
3,3,0.510360,0.530695,0.510877,0.524348,0.526178,0.512709,0.521005
4,4,0.612933,0.635392,0.502321,0.349346,0.665423,0.473886,0.568218
